# Ignore a tracked file


**`.gitignore` only ignores untracked files**. 


[Untrack all files and then add again after exclude some files in `.gitignore`](https://stackoverflow.com/a/36394436/5186966) - if you don't mind others that pull from your repo afterwards, have their files using the same names deleted in their hard drive.


[Difference among `.gitignore`, `.git/info/exclude` and `assume-unchanged` explained by a Git maintainer](https://stackoverflow.com/a/23305143/5186966)


# Removing Files

|File Status|Impacts|Command|Use Case|
|:---|:---|:---|:---|
|Untracked file|Remove from working directory|`rm <file>`|Delete the file before it's ever tracked|
|Tracked file|Remove from both staging area and working directory|`git rm <file>`|Delete the file completely|
|Tracked file|Remove from staging area (not to delete from working directory)|`git rm --cached <file>`|Keep the file in working tree but not to have Git track it anymore <br>**Beware** if this repo is pulled by other developer, the file using the same name on their working directory will be deleted too)|
|

However, none of the above will remove the file data from showing in the commits history, if these files contain sensitive data that shall be removed completely, use [`git filter-branch` or `bfg repo cleaner tool`](https://help.github.com/en/github/authenticating-to-github/removing-sensitive-data-from-a-repository)

# Remove Git version control from a project folder

```Shell
rm -rf .git
rm -rf .gitkeep
```

# How to store dotfiles 

## Use bare Git repo without symlinks 

**Not recommended** - `git clean` will erase everything in `~`

[Tutorial - store your dotfiles: a bare Git repo ](https://www.atlassian.com/git/tutorials/dotfiles)
> - [Why use a bare repo for dotfiles `git init --bare`](http://www.saintsjd.com/2011/01/what-is-a-bare-git-repository/)
> - No extra tooling, no symlinks, 
> - files are tracked on VCS
> - you can use different branches for different computers 
> - you can replicate you configuration easily on new installation


## My preferred way

### Directory and symlinks

- Save dotfiles in `~/.dotfiles`
- use symlinks to link the dotfiles to the system paths where expected, for example, `~/.dotfiles/.zshrc` shall have the symlink to `~/.zshrc` (The symlinks creation scripts are stored in `install.sh`)


### Github repos

- Use Git to track `~/.dotfiles` and store it in Github repos. 

- I will have one private Github repo that stores everything, and one public Github repo that exclude the personal files for public sharing.

- The private Github repo connects to my local `master` branch, and public Github repo connects to local `public` branch of `~/.dotfiles`



### Keep different `.gitignore` files in different branches

- [How to keep different `.gitignore` files for different branches](https://community.atlassian.com/t5/Bitbucket-questions/How-to-set-up-a-branch-specific-ignoring-pattern/qaq-p/150257)

> The `.gitignore` file being tracked like any other is precisely what allows it to be different in different branches, i.e. whenever you check out a branch, the `.gitignore` will be replaced by the one in that branch and the right version will be used when committing changes. Here are the steps to do so:
1. You create a `.gitignore` in your working folder and commit it to the branch you have currently checked out. 
2. You switch to another branch. (The `.gitignore` file will be gone from your working folder.
3. You create a new, different `.gitignore` in your working folder and commit it to the branch you have currently checked out. 
4. Repeat 2 and 3 as desired for other branches. If you create branches after this is done, the new branch will, at the beginning, have the same `.gitignore` as the parent branch, but you can make edits and commit them, as needed.


### Make sure personal files in `master` NEVER merged into `public` branch

Although with `.gitignore` file in place, we have to also make sure the personal files in `master` NEVER be tracked in `public` because of merging `master` into `public`, because
>- `.gitignore` can only ignore untracked files. 
>- A ff merge will add the ignored files into another branch.
>- Also, I don't want the personal files appear in the commit history, otherwise, it will lose the purpose of hiding my personal data. so `git rm --cached` won't help. 
>    - `git filter-branch` won't help in my case, because this command work on the whole repo, it will also remove the personal files from the history of the other branch.


So, I need to find a way to prevent personal files being added in a ff merge automatically. There are 2 ways of doing so:

1. [Do the merge without automatic commits and fast forward](https://stackoverflow.com/a/16455853/5186966) 
    - `git merge --no-ff --no-commit` to merge without commit first even if it's a ff merge, and then remove/unstage unwanted files, then conclude the merge.
    - [How to do the interactive unstaging](https://git-scm.com/book/en/v2/Git-Tools-Interactive-Staging#Merge-Strategies)


2. [Use `.gitattribute` file and the config `merge.ours.driver` as `True`](https://medium.com/@porteneuve/how-to-make-git-preserve-specific-files-while-merging-18c92343826b) - **RECOMMENDED, TESTED SUCCESSFULLY*
    - There is also a way of [keeping `.gitattribute` locally without being tracked](https://stackoverflow.com/a/14099431/5186966) 



### PENDING

My local dotfiles repo has 2 branches: `master` and `private`. I push `master` to a private Github repo and `private` to public Github repo. My goals include:

1. I can merge updates in `master` to `public`

2. I can ignore the personal files in `private` branch and make sure it won't be impacted by the merge from `master`

Now my setup is as follows:

- I have 2 different `.gitignore` in `master` and `public` separately ([how to do this](https://community.atlassian.com/t5/Bitbucket-questions/How-to-set-up-a-branch-specific-ignoring-pattern/qaq-p/150257)) and passed tests successfully

- I have used [this method](https://medium.com/@porteneuve/how-to-make-git-preserve-specific-files-while-merging-18c92343826b) to make sure `.gitignore` in `public` won't be overwritten by the one in `master` when merge, even in a fast-forward merge.

However, this still can't help solve my problem. For example, I don't want to have my `.gitconfig` with personal email information in public repo, so this file is added in `.gitignore` of `public` branch. However, when I merge `master` into `public`, `.gitconfig` will merge in from `master` and it will be committed to conclude the merge, so `.gitignore` cannot work on this tracked file anymore. Even though I `git rm` this file, it's still visible in the commit history.

I also tried another way, using [this method](https://medium.com/@porteneuve/how-to-make-git-preserve-specific-files-while-merging-18c92343826b) to keep two versions of `.gitconfig` in different branches and prevent the content of `.gitconfig` in one branch being overwritten by another, like what I did with `.gitignore`. However, because merge will have history of the commits of both versions show, it just cannot achieve the purpose of hiding personal data.

So I just want to know is there any other way to do what I want? Does anyone think this is actually achievable? I read some post suggesting writing a `post-checkout hook`. But that's beyond my knowledge, I'd need some guidance on this, if any mentor can help.